In [13]:
## !pip install mlxtend
## !pip install onnxmltools
## !pip install onnxruntime
## !pip install torch torchvision
## !pip install skl2onnx

     -------------------------------------- 300.3/300.3 kB 1.2 MB/s eta 0:00:00
     ------------------------------------ 298.4/298.4 kB 878.6 kB/s eta 0:00:00
     -------------------------------------- 298.5/298.5 kB 1.1 MB/s eta 0:00:00


In [4]:
import numpy as np
import torch
import pandas as pd
import sklearn
import random

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [46]:
import matplotlib.pyplot as plt

from mlxtend.plotting import heatmap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import TensorDataset, DataLoader

## coefficient of determination 
from sklearn.metrics import r2_score

In [ ]:
import xgboost as xgb

In [14]:
import onnxruntime as rt
import onnxmltools

from skl2onnx.common.data_types import FloatTensorType

In [15]:
## 0.001, 0.0001, 0.0003, 0.01, 0.03

batch_size    = 32
learning_rate = 0.005 ## 0.001
N_Epochs      = 100

epsilon = 0.0001

In [18]:
path_data = 'WeatherData.csv'

Weather_raw_data = pd.read_csv( path_data, delimiter="," )


In [19]:

Weather_raw_data

,Location,Temperature_C,Humidity_pct,Wind_Speed_kmh,Date_Time
0,Philadelphia,-8.63,54.07,26.37,2/26/2024 17:32
1,New York,-6.89,30.80,16.85,1/25/2024 19:04
2,New York,0.96,38.82,7.99,3/29/2024 5:20
3,New York,35.15,54.75,25.43,3/4/2024 13:47
4,Dallas,32.02,53.19,3.05,2/27/2024 21:07
...,...,...,...,...,...
495,Dallas,35.35,66.51,16.81,4/6/2024 3:57
496,Los Angeles,33.80,40.55,10.46,5/15/2024 4:28
497,Chicago,19.86,45.88,27.25,4/9/2024 4:35
498,Philadelphia,21.00,61.12,12.01,5/8/2024 5:39


In [20]:
headers_list = Weather_raw_data.columns.values.tolist()

headers_list


['Location', 'Temperature_C', 'Humidity_pct', 'Wind_Speed_kmh', 'Date_Time']

In [22]:
Weather_raw_data_np = Weather_raw_data.to_numpy()

In [23]:
Weather_raw_data_np

array([['Philadelphia', -8.63, 54.07, 26.37, '2/26/2024 17:32'],
       ['New York', -6.89, 30.8, 16.85, '1/25/2024 19:04'],
       ['New York', 0.96, 38.82, 7.99, '3/29/2024 5:20'],
       ...,
       ['Chicago', 19.86, 45.88, 27.25, '4/9/2024 4:35'],
       ['Philadelphia', 21.0, 61.12, 12.01, '5/8/2024 5:39'],
       ['Los Angeles', -3.04, 49.09, 28.33, '2/6/2024 18:00']],
      dtype=object)

In [24]:
Weather_raw_data_np.shape

(500, 5)

In [110]:
X = Weather_raw_data_np[:, 1:4]     
y = Weather_raw_data_np[:, 0]
time_column = Weather_raw_data_np[:, 4]

In [111]:

y

array(['Philadelphia', 'New York', 'New York', 'New York', 'Dallas',
       'Philadelphia', 'Chicago', 'New York', 'New York', 'Los Angeles',
       'New York', 'Dallas', 'Chicago', 'Philadelphia', 'Dallas',
       'Philadelphia', 'Chicago', 'New York', 'Dallas', 'New York',
       'Philadelphia', 'Chicago', 'Chicago', 'Philadelphia', 'New York',
       'New York', 'Philadelphia', 'Chicago', 'Dallas', 'Los Angeles',
       'Chicago', 'Los Angeles', 'New York', 'New York', 'New York',
       'Los Angeles', 'Dallas', 'Dallas', 'Chicago', 'Philadelphia',
       'Los Angeles', 'Dallas', 'Los Angeles', 'Los Angeles',
       'Philadelphia', 'Philadelphia', 'Chicago', 'Philadelphia',
       'Chicago', 'Chicago', 'New York', 'Dallas', 'New York', 'Dallas',
       'Dallas', 'Los Angeles', 'New York', 'Chicago', 'Los Angeles',
       'Los Angeles', 'Chicago', 'Dallas', 'Chicago', 'Philadelphia',
       'Dallas', 'Los Angeles', 'Dallas', 'Chicago', 'Los Angeles',
       'Philadelphia', 'Los Angel

In [112]:
print(X.shape)

print(y.shape)

encoder = LabelEncoder()
y_encoder = encoder.fit_transform(y) 

(500, 3)
(500,)


In [113]:
random_seed = int( random.random() * 100 )     ## 42

In [114]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoder, test_size=0.2, random_state=random_seed)

In [115]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(400, 3)
(100, 3)
(400,)
(100,)


In [203]:
y_train.dtype 

dtype('float32')

In [124]:
X_train = X_train.astype(  np.float32  )
X_test  = X_test.astype(   np.float32  )
y_train = y_train.astype(  np.float32  )
y_test  = y_test.astype(   np.float32  )

In [125]:
X_train_tr = torch.from_numpy(X_train)
X_test_tr  = torch.from_numpy(X_test)
y_train_tr = torch.from_numpy(y_train)
y_test_tr  = torch.from_numpy(y_test)

In [126]:
x_means      = X_train_tr.mean(0, keepdim=True ) 

x_deviations = X_train_tr.std( 0, keepdim=True) + epsilon

In [127]:
x_means


tensor([[13.8017, 57.9944, 14.9805]])

In [128]:
x_deviations

tensor([[14.5332, 18.1744,  8.7409]])

In [129]:
train_ds = TensorDataset( X_train_tr, y_train_tr  )

In [130]:
train_dl = DataLoader( train_ds, batch_size, shuffle=True  )

In [131]:
train_dl

In [132]:
import torch
import torch.nn as nn

class DL_Net(nn.Module):
    ## init the class
    def __init__(self, x_means, x_deviations):
        super().__init__()
        
        self.x_means      = x_means
        self.x_deviations = x_deviations
        
        self.linear1 = nn.Linear(3, 10)  # Input size is 3
        self.act1    = nn.ReLU()
        self.linear2 = nn.Linear(10, 6)
        self.act2    = nn.ReLU()
        self.linear3 = nn.Linear(6, 1)  # Output size is 1 for regression
        self.dropout = nn.Dropout(0.25)
        
    ## perform inference
    def forward(self, x):
        x = (x - self.x_means) / self.x_deviations  # Standardizing input
        
        x = self.linear1(x)
        x = self.act1(x)
        x = self.linear2(x)
        x = self.act2(x)
        # x = self.dropout(x)
        y_pred = self.linear3(x)
        
        
        return y_pred


In [133]:
def training_loop( N_Epochs, model, loss_fn, opt  ):
    
    for epoch in range(N_Epochs):
        for xb, yb in train_dl:
            
            y_pred = model(xb)
            loss   = loss_fn(y_pred, yb)
            
            opt.zero_grad()
            loss.backward()
            opt.step()
            
        if epoch % 20 == 0:
            print(epoch, "loss=", loss)
            
        

In [134]:
model = DL_Net( x_means, x_deviations  )

opt     = torch.optim.Adam(    model.parameters(), lr=learning_rate )
loss_fn = F.mse_loss

training_loop(  N_Epochs, model, loss_fn, opt  )

C:\Users\21tim\anaconda3\envs\py37_ITS365\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  import sys


0 loss= tensor(2.4635, grad_fn=<MseLossBackward0>)
20 loss= tensor(1.3135, grad_fn=<MseLossBackward0>)
40 loss= tensor(1.8857, grad_fn=<MseLossBackward0>)
60 loss= tensor(1.6257, grad_fn=<MseLossBackward0>)
80 loss= tensor(1.9558, grad_fn=<MseLossBackward0>)


In [135]:
y_pred_test = model( X_test_tr )

In [136]:
y_pred_test.shape
## y_pred_test

torch.Size([100, 1])

In [137]:
print( "Testing R**2: ", r2_score(  y_test_tr.numpy(),  y_pred_test.detach().numpy()     )  )

Testing R**2:  -0.005849751646710866


In [138]:
y_pred_test.shape

torch.Size([100, 1])

In [139]:
y_test_tr.shape
y = 

torch.Size([100])

In [140]:
len(X_test_tr)

100

In [142]:
list_preds = []
list_reals = []

for i in range(len(X_test_tr)):
    print("************************************")
    print("pred, real")
    np_real =   y_test_tr[i].detach().numpy()
    np_pred = y_pred_test[i].detach().numpy()
    print(( np_pred  , np_real))
    list_preds.append(np_pred[0])
    list_reals.append(np_real[0])

************************************
pred, real
(array([2.0497344], dtype=float32), array(4., dtype=float32))


IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

In [ ]:
model.eval()

dummy_input = torch.randn(1, 3)

input_names  = ["input1"]
output_names = ["output1"]

torch.onnx.export(
        model, 
        dummy_input,
        "DLnet_WeatherData.onnx",
        verbose=False,
        input_names = input_names,
        output_names = output_names
)